In [3]:
!pip install vaderSentiment
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import math
from datetime import datetime, timedelta
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import math


historical_reddit_data = pd.read_csv('subredditdata.csv')
historical_prices_data = pd.read_csv('historicalprices.csv')

def breakup_by_date(data):
    breakpoints= []
    dataframes = []
    current_index_time = datetime(2018,7,24,16,55)
    new_date = current_index_time
    target_date = new_date + timedelta(days=1)
        
    for index,val in enumerate(data['CREATED_UTC'].to_list()):
         if datetime.strptime(val, '%Y-%m-%d %H:%M:%S') < target_date:
             continue
         else:
             breakpoints.append(index)
             target_date = target_date + timedelta(days=1)
             
    for index, item in enumerate(breakpoints):
        if index == 0:
            abridged_dataframe = data.iloc[0:item]
            dataframes.append(abridged_dataframe)
        
        else:
            abridged_dataframe = data.iloc[breakpoints[index-1]:item]
            dataframes.append(abridged_dataframe)
            
    return dataframes
        

              
    
dataframes = breakup_by_date(historical_reddit_data)

dataframes.pop(-1)

analyzer = SentimentIntensityAnalyzer()

def generate_prices_delta(pricesdata):
    delta_list = []
    pricelist= pricesdata['CLOSE'].iloc[570:].to_list()
    for index, entry in enumerate(pricelist):
        try:
            pricedelta = pricelist[index+1] - entry
            delta_list.append(pricedelta)
        except IndexError: 
            break
    return delta_list
    
hmm = historical_prices_data['CLOSE'].iloc[570:].to_list()  
outputs = generate_prices_delta(historical_prices_data)             
                
        

def prepare_data(data):
    new_dataframes = []
    for dataframe in data:
        dataframe['compound'] = [analyzer.polarity_scores(x)['compound'] for x in dataframe['TITLE']]
        dataframe['neg'] = [analyzer.polarity_scores(x)['neg'] for x in dataframe['TITLE']]
        dataframe['neu'] = [analyzer.polarity_scores(x)['neu'] for x in dataframe['TITLE']]
        dataframe['pos'] = [analyzer.polarity_scores(x)['pos'] for x in dataframe['TITLE']]
        new_dataframes.append(dataframe)
    return new_dataframes

lol = prepare_data(dataframes)

def prepare_input_vector(data):
    vectors = []
    for item in data:
        avg_compound = item['compound'].mean()
        avg_neg = item['neg'].mean()
        avg_neu = item['neu'].mean()
        avg_pos = item['pos'].mean()
        number_of_posts = len(item.index)
        vector = [avg_compound, avg_neg, avg_pos, avg_neu, number_of_posts]
        vectors.append(vector)
    return vectors

to_feed = prepare_input_vector(lol)









/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    to_feed, outputs, test_size=0.33, random_state=42)

In [6]:
#input_A = keras.layers.Input(shape=[4],name='numerical_input')
#hidden_1= keras.layers.Dense(250, activation = 'sigmoid')(input_A)
#hidden_2= keras.layers.Dense(500, activation = 'sigmoid')(hidden_1)
#hidden_3= keras.layers.Dense(350, activation = 'sigmoid')(hidden_2)
#hidden_4= keras.layers.Dense(150, activation = 'sigmoid')(hidden_3)
#hidden_5= keras.layers.Dense(20, activation = 'sigmoid')(hidden_4)
#output = keras.layers.Dense(1, name = 'output', activation= )(hidden_5)
#model = keras.Model(inputs=input_A, outputs=output)

model = keras.Sequential(
    [
        layers.Dense(5, activation="relu", name="layer1"),
        #layers.Dense(1000, activation="relu", name="layer2"),
        #layers.Dense(1000, activation="relu", name="layer3"),
        #layers.Dense(1000, activation="relu", name="layer4"),
        #layers.Dense(1000, activation="relu", name="layer5"),
        #layers.Dense(300, activation="relu", name="layer6"),
        layers.Dense(250, activation="relu", name="layer7"),
        layers.Dense(120, activation="relu", name="layer8"),
        layers.Dense(1, name="output", activation='linear'),
    ]
)

optimizer = keras.optimizers.Adagrad(learning_rate= .1, initial_accumulator_value=5)

model.compile(loss='mean_absolute_percentage_error', optimizer= optimizer, metrics=['accuracy'])


leng = len(y_train)

def knockout_nan_rows(x,y):
  for i in range(len(x)-1):
    if math.isnan(y[i]):
      x.pop(i)
      y.pop(i)
  return x, y
  


X_train, y_train = knockout_nan_rows(X_train, y_train)

X_train = np.array(X_train)

y_train = np.array(y_train)









history = model.fit(X_train, y_train, epochs = 20000, validation_split = 0.4)

weights = model.get_weights()



Streaming output truncated to the last 5000 lines.
15/15 [==============================] - 0s 6ms/step - loss: 92.9758 - accuracy: 0.0000e+00 - val_loss: 131.5597 - val_accuracy: 0.0000e+00
Epoch 17502/20000
15/15 [==============================] - 0s 6ms/step - loss: 93.0586 - accuracy: 0.0000e+00 - val_loss: 131.9501 - val_accuracy: 0.0000e+00
Epoch 17503/20000
15/15 [==============================] - 0s 6ms/step - loss: 93.0800 - accuracy: 0.0000e+00 - val_loss: 131.4912 - val_accuracy: 0.0000e+00
Epoch 17504/20000
15/15 [==============================] - 0s 9ms/step - loss: 93.0186 - accuracy: 0.0000e+00 - val_loss: 131.2334 - val_accuracy: 0.0000e+00
Epoch 17505/20000
15/15 [==============================] - 0s 10ms/step - loss: 93.0503 - accuracy: 0.0000e+00 - val_loss: 131.6793 - val_accuracy: 0.0000e+00
Epoch 17506/20000
15/15 [==============================] - 0s 6ms/step - loss: 93.0519 - accuracy: 0.0000e+00 - val_loss: 131.3809 - val_accuracy: 0.0000e+00
Epoch 17507/20000


In [ ]:
print(y_train)

In [7]:
model.predict(X_test)


array([[  -0.43268928],
       [  -0.43268928],
       [  -0.43268928],
       [  -0.43268928],
       [  -2.0605795 ],
       [  -0.43268928],
       [ 206.01503   ],
       [  -0.43268928],
       [ -43.02788   ],
       [  10.433977  ],
       [  -0.43268928],
       [  10.595242  ],
       [   6.7786613 ],
       [   8.000925  ],
       [  -2.0605795 ],
       [  -0.43268928],
       [   6.8878284 ],
       [   6.976902  ],
       [  10.595242  ],
       [  -0.43268928],
       [  -2.0605795 ],
       [  -0.43268928],
       [  -0.43268928],
       [  -0.43268928],
       [  -0.43268928],
       [  -0.43268928],
       [  -0.43268928],
       [  -0.43268928],
       [  -9.304899  ],
       [  37.047276  ],
       [  -0.43268928],
       [  -2.0605776 ],
       [  -2.0605795 ],
       [  -0.43268928],
       [  -0.43268928],
       [  -0.43268928],
       [  -0.43268928],
       [  -0.43268928],
       [  -0.43268928],
       [  -0.43268928],
       [  -0.43268928],
       [ -44.116

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [11]:
import torch

model.save('team_3_model.m5')

INFO:tensorflow:Assets written to: team_3_model.m5/assets
